In [3]:
import numpy as np
from math import cos,sin,pi
from itertools import combinations

In [2]:
 def Rx(theta):
    return np.matrix([[ 1, 0         , 0         ],
                      [ 0, cos(theta),-sin(theta)],
                      [ 0, sin(theta), cos(theta)]])
  
def Ry(theta):
    return np.matrix([[ cos(theta), 0, sin(theta)],
                      [ 0         , 1, 0         ],
                      [-sin(theta), 0, cos(theta)]])
  
def Rz(theta):
    return np.matrix([[ cos(theta), -sin(theta), 0 ],
                      [ sin(theta),  cos(theta), 0 ],
                      [ 0         , 0          , 1 ]])

def R(psi,theta,phi):
     return Rz(psi) * Ry(theta) * Rx(phi)

def getRotations():
    Rotations = []
    for i in range(4):
        for j in range(4):
            for k in range(4):
                phi   = i*np.pi/2
                theta = j*np.pi/2
                psi   = k*np.pi/2
                Rot = np.array(np.round(R(psi,theta,phi),decimals=2),dtype=int)
                unique = True
                for Rot0 in Rotations:
                    if (Rot==Rot0).all():
                        unique = False
                if unique:
                    Rotations.append(Rot)
    return Rotations

Rotations = getRotations()
print("Generated {} rotation matrices".format(len(Rotations)))

Generated 24 rotation matrices


In [4]:
def dist(a, b):
    return sum([(p_a-p_b)*(p_a-p_b) for p_a, p_b in zip(a, b)])

class Scanner():
    
    def __init__(self, idnum):
        self.idnum = idnum
        self.beacons = []
        self._orientation = 0
        self.offset = (0, 0, 0)
        self.checked = []
        self.distances = []
        self.pairs = []
        
    def add_beacon(self, b):
        if b not in self.beacons:
            self.beacons.append(b)
                             
    def calc_distances(self):
        self.pairs = [(a, b) for a, b in combinations(self.beacons, 2)]
        self.distances = [dist(a, b) for a, b in combinations(self.beacons, 2)]

    def set_offset(self, offset):
        self._offset = offset
        
    def set_orientation(self, o):
        self._orientation = o
        
    def coords(self, perm, rot):
        c = np.stack(self.beacon_x, self._beacon_y, self._beacon_z)
        
        return c[perm[0]]*rot[0], c[perm[1]]*rot[1], c[perm[2]]*rot[2]

In [5]:
def read_data(fname):
    scanners = []
    n = 0

    with open(fname, 'r') as inf:
        for line in inf.readlines():
            if line.startswith('--'):
                scanner = Scanner(n)
                n += 1
            elif line.strip() == '':
                scanners.append(scanner)
            else:
                scanner.add_beacon(tuple([int(i) for i in line.strip().split(',')]))
        scanners.append(scanner)

    for s in scanners:
        s.calc_distances()
    return scanners

In [6]:
def overlap(sc1, sc2):
    common1, common2 = [], []
    common = list(set(sc1.distances).intersection(set(sc2.distances)))
    if len(common) >= 66:
        for c in common:
            idx1 = sc1.distances.index(c)
            idx2 = sc2.distances.index(c)
            for p in sc1.pairs[idx1]:
                if p not in common1:
                    common1.append(p)
            for p in sc2.pairs[idx2]:
                if p not in common2:
                    common2.append(p)
    return common1, common2

In [7]:
def match_scanners(sc1, sc2):
    c1, c2 = overlap(sc1, sc2)
    if c1 == []:
        print('No overlap for scanners', sc1.idnum, sc2.idnum)
        return
    
    p1 = np.array(c1[0])
    found = False
    
    for p2 in c2:
        p2 = np.array(p2)
        for R in Rotations:
            p2_rot = np.matmul(R, p2)
            d = p1 - p2_rot
            
            c2_trans = [ tuple(np.matmul(R, np.array(p_t))+d) for p_t in c2]
            in_ref = [1 for p_t in c2_trans if p_t in c1]
            
            if len(in_ref) >= 12:
                print('Matched', sc1.idnum, sc2.idnum)
                found = True
                break
        if found:
            break
    if found:
        new_scanner = Scanner(sc2.idnum)
        new_scanner.beacons = [tuple(np.matmul(R, np.array(p2)) + d) for p2 in sc2.beacons]
        new_scanner.offset = d
        return new_scanner
    return None

In [8]:
scanners = read_data('input19.txt')

ref_scanner = scanners[0]
known_scanners = [scanners[0]]
not_found = [i for i in range(1, len(scanners))]

while len(not_found) > 0:
    for i in range(1, len(scanners)):
        if i in not_found:
            matched = match_scanners(ref_scanner, scanners[i])
            if matched is not None:
                del(not_found[not_found.index(i)])
                for b in matched.beacons:
                    ref_scanner.add_beacon(b)
                ref_scanner.calc_distances()
                known_scanners.append(matched)
    print('Unknown scanners', not_found)
print(len(ref_scanner.beacons))

No overlap for scanners 0 1
No overlap for scanners 0 2
No overlap for scanners 0 3
No overlap for scanners 0 4
No overlap for scanners 0 5
No overlap for scanners 0 6
No overlap for scanners 0 7
No overlap for scanners 0 8
No overlap for scanners 0 9
No overlap for scanners 0 10
No overlap for scanners 0 11
No overlap for scanners 0 12
No overlap for scanners 0 13
No overlap for scanners 0 14
No overlap for scanners 0 15
No overlap for scanners 0 16
No overlap for scanners 0 17
No overlap for scanners 0 18
No overlap for scanners 0 19
No overlap for scanners 0 20
No overlap for scanners 0 21
No overlap for scanners 0 22
No overlap for scanners 0 23
No overlap for scanners 0 24
No overlap for scanners 0 25
No overlap for scanners 0 26
No overlap for scanners 0 27
No overlap for scanners 0 28
No overlap for scanners 0 29
No overlap for scanners 0 30
No overlap for scanners 0 31
No overlap for scanners 0 32
No overlap for scanners 0 33
No overlap for scanners 0 34
No overlap for scanners

In [9]:
def manhattanDistance(a,b):
    return sum([abs(xa-xb) for xa,xb in zip(a,b)])

offsets = [sc.offset for sc in known_scanners]

dmax = max([manhattanDistance(a, b) for a, b in combinations(offsets, 2)])

print(dmax)

14804
